# Charger le modèle depuis le fichier .h5

In [ ]:
model = # A COMPLETER
print("Modèle chargé avec succès.")

# Vérifier la structure du modèle pour identifier la dernière couche de convolution

In [ ]:
model.summary()

# Générer la carte de chaleur Grad-CAM

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    # Obtenir le modèle pour la carte de chaleur Grad-CAM
    grad_model = Model(
        [model.inputs], 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Calculer le gradient de la classe prédite par rapport à la dernière couche de convolution
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    grads = tape.gradient(loss, conv_outputs)

    # Calculer la moyenne des gradients le long de l'axe de profondeur (canal)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiplier chaque canal par son poids moyen
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    # Appliquer ReLU pour ne conserver que les valeurs positives
    heatmap = tf.maximum(heatmap, 0)

    # Normaliser la carte de chaleur
    heatmap /= tf.reduce_max(heatmap)
    return heatmap.numpy()

# Fonction pour superposer la carte de chaleur sur l'image originale

In [ ]:
def superimpose_heatmap(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return superimposed_img

# Test

In [ ]:
# Charger une image de test
image_path = random.choice(random_images)
img = tf.keras.utils.load_img(image_path, target_size=(256, 256))
img_array = tf.keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0
prediction = model.predict(img_array)

# Générer la carte de chaleur
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name="conv2d_2")

# Superposer la carte de chaleur sur l'image originale
superimposed_img = superimpose_heatmap(np.array(img, dtype=np.uint8), heatmap)

# Afficher l'image avec la carte de chaleur
plt.figure(figsize=(10, 10))
plt.imshow(superimposed_img)
plt.title("Carte de chaleur Grad-CAM")
plt.axis("off")
plt.show()